In [1]:
import numpy as np

# Problem 8
$$ - \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) - e^u = 0$$
Leads to the FD equation
$$4u_{i,j} - u_{i-1,j} - u_{i+1,j} - u_{i,j-1} - u_{i,j+1} - h^2e^{u_{i,j}} = 0$$
for $ 0< i,j < n$
$$f(\vec{u}) = A\vec{u} - e^\vec{u} = 0$$
So the Jacobian is given by the following formulae
$$
\frac{\partial f}{\partial u_{i,j}} = 4 - h^2e^{u_{i,j}}
$$